In [4]:
KeyPath = %env CODINGDOJO

In [5]:
import json
with open(KeyPath) as f: 
    login = json.load(f)
login.keys()

dict_keys(['MySQL', 'yelp-id', 'yelp-api-key'])

In [ ]:
## save dict to json file with json.dump
with open('example_saved_data.json','w') as f:
    json.dump(data, f)

